#### 经验熵：$H(x) = -\sum_{i=1}^{n}p_i\log{p_i}$

#### 经验条件熵: $H(X|Y)=\sum{P(X|Y)}\log{P(X|Y)}$

#### 信息增益 : $g(D, A)=H(D)-H(D|A)$

#### 信息增益比: $g_R(D, A) = \frac{g(D,A)}{H(A)}$

#### 基尼指数:$Gini(D)=\sum_{k=1}^{K}p_k\log{p_k}=1-\sum_{k=1}^{K}p_k^2$

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# 题目5.1
def create_data():
    dataset = [['青年', '否', '否', '一般', '否'],
               ['青年', '否', '否', '好', '否'],
               ['青年', '是', '否', '好', '是'],
               ['青年', '是', '是', '一般', '是'],
               ['青年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '好', '否'],
               ['中年', '是', '是', '好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '好', '是'],
               ['老年', '是', '否', '好', '是'],
               ['老年', '是', '否', '非常好', '是'],
               ['老年', '否', '否', '一般', '否'],
               ]
    labels = [u'年龄', u'有工作', u'有自己的房子', u'信贷情况', u'类别']
    # 返回数据集和每个维度的名称
    return dataset, labels
dataset, labels = create_data()
train_data = pd.DataFrame(dataset, columns=labels)

In [3]:
train_data

,年龄,有工作,有自己的房子,信贷情况,类别
0,青年,否,否,一般,否
1,青年,否,否,好,否
2,青年,是,否,好,是
3,青年,是,是,一般,是
4,青年,否,否,一般,否
5,中年,否,否,一般,否
6,中年,否,否,好,否
7,中年,是,是,好,是
8,中年,否,是,非常好,是
9,中年,否,是,非常好,是


In [20]:
train_data['有工作'].unique()

array(['否', '是'], dtype=object)

---  
用C4.5算法来生成树

In [58]:
class Node:
    def __init__(self, leaf=True, label=None, feature_name=None, feature_num=None):
        self.leaf = leaf
        self.label = label
        self.feature_name = feature_name
        self.feature_num = feature_num
        self.tree = {}
        
    def add_node(self, val, node):
        self.tree[val] = node
        
    def predict(self, features):
        if self.leaf is True:
            return self.label
        return self.tree[features[self.feature_num]].predict(features)

In [77]:
class DTree:
    def __init__(self, epsilon=0.1):
        self.epsilon = epsilon
        self._tree = {}
    
    #经验熵
    def entropy(self, dataset, axis=-1):
        N = len(dataset)
        label_count = {}
        for i in range(N):
            label = dataset[i][axis]
            if label not in label_count:
                label_count[label] = 0
            label_count[label] += 1
        entropy = -sum([cla/N * np.log(cla/N) for cla in label_count.values()])
        return entropy
    
    #条件经验熵
    def cond_entropy(self, dataset, feature_num=0):
        N = len(dataset)
        features_set = {}
        for i in range(N):
            feature = dataset[i][feature_num]
            if feature not in features_set:
                features_set[feature] = []
            features_set[feature].append(dataset[i])
        cond_entropy = sum([len(n)/N * self.entropy(n) for n in features_set.values()])
        return cond_entropy
    
    #信息增益比
    def info_gain_rate(self, dataset, feature_num):
        info_gain = self.entropy(dataset) - self.cond_entropy(dataset, feature_num)
        HaD = self.entropy(dataset, feature_num)
        return info_gain / HaD
    
    #各feature的信息增益比
    def info_gain_rate_count(self, dataset):
        feature_count = len(dataset[0]) - 1
        feature_list = []
        for num in range(feature_count):
            feature_info_gain_rate = self.info_gain_rate(dataset, num) 
            feature_list.append((num,feature_info_gain_rate))
#             print('特征({})的信息增益比为：{:.3f}'.format(labels[num],feature_info_gain_rate))
        best_feature = max(feature_list, key=lambda x:x[-1])
#         print('特征({})的信息增益比最大'.format(labels[best_feature[0]]))
        return best_feature
    
    def train(self, train_data):
        """
        input:DataFrame
        """
        X_train, y_train, features = train_data.iloc[:,:-1], train_data.iloc[:,-1], train_data.columns[:-1]
        
        #停止条件：当数据都属于同一类时
        if len(y_train.value_counts()) == 1:
            return Node(leaf=True, label=y_train.iloc[0])
        
        #停止条件：当feature数为0时
        if len(features) == 0:
            return Node(leaf=True, label=y_train.value_counts().sort_values()[-1])
        
        #计算最大的信息增益比,得到最佳的feature
        best_feature, max_rate = self.info_gain_rate_count(np.array(train_data))
        best_feature_name = features[best_feature]
        
        #停止条件：当信息增益比小于阈值eta时
        if max_rate < self.epsilon:
            return Node(leaf=True, label=y_train.value_counts().sort_values()[-1])
        
        #构建子树
        node_tree = Node(leaf=False, feature_name=best_feature_name, feature_num=best_feature )
        
        feature_values = train_data[best_feature_name].unique()
        for v in feature_values:
            sub_train_df = train_data.loc[train_data[best_feature_name] == v].drop([best_feature_name], axis=1)
            sub_tree = self.train(sub_train_df)
            node_tree.add_node(v, sub_tree)
        return node_tree
    
    def fit(self, train_data):
        self._tree = self.train(train_data)
        return self._tree
    
    def predict(self, X_test):
        return self._tree.predict(X_test)

In [78]:
dt = DTree()

In [81]:
tree = dt.fit(train_data)

In [85]:
dt.predict(['老年', '否', '是', '一般'])

'是'